In [106]:
import requests
import json
import os
import pandas as pd
from functools import reduce
import operator
import pyjsonviewer
import geopandas as gpd
import time
import requests
from bs4 import BeautifulSoup
from shapely.geometry import Point
from pymongo import MongoClient
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from pymongo import GEOSPHERE
import shapely.geometry

from dotenv import load_dotenv
load_dotenv()

True

In [107]:
df_o = pd.read_csv('../data/misOfis.csv')
df_e = pd.read_csv('../data/empresasTech.csv')
df_l = pd.read_csv('../data/df_lugares.csv')

In [108]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [109]:
mo = db.get_collection("misOfis")
me = db.get_collection("misEmp")
ml = db.get_collection("misLug")

In [110]:
geomo = gpd.GeoDataFrame(df_p, geometry=gpd.points_from_xy(df_p.longt, df_p.latt))
geome = gpd.GeoDataFrame(df_e, geometry=gpd.points_from_xy(df_e.longitude, df_e.latitude))
geoml = gpd.GeoDataFrame(df_l, geometry=gpd.points_from_xy(df_l.longitud, df_l.latitud))

In [111]:
geomo['geometry']=geomo['geometry'].apply(lambda x:shapely.geometry.mapping(x))
geomodic = geomo.to_dict(orient="records")
mo.create_index([("geometry", GEOSPHERE)])
mo.insert_many(geomodic)

/home/fer/miniconda3/envs/minimal_ds/lib/python3.9/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [112]:
geome['geometry']=geome['geometry'].apply(lambda x:shapely.geometry.mapping(x))
geomedic = geome.to_dict(orient="records")
me.create_index([("geometry", GEOSPHERE)])
me.insert_many(geomodic)

/home/fer/miniconda3/envs/minimal_ds/lib/python3.9/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [113]:
geoml['geometry']=geoml['geometry'].apply(lambda x:shapely.geometry.mapping(x))
geomldic = geoml.to_dict(orient="records")
ml.create_index([("geometry", GEOSPHERE)])
ml.insert_many(geomodic)

/home/fer/miniconda3/envs/minimal_ds/lib/python3.9/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [122]:
unod = df_o['coord'][0]
unod[30:]


"': [40.39238, -3.68972]}"

In [127]:
querymo = [{"$geoNear": {
            "near": [ -3.68972,40.39238],
            "distanceField": "geodistancia",
            "maxDistance": 1000,
            "distanceMultiplier" :6371,
            "spherical": True}}]
geoloc_mo = list(mo.aggregate(querymo))

In [128]:
geoloc_mo = pd.DataFrame(geoloc_mo)
geoloc_mo.drop(['_id','geometry'], axis=1, inplace=True)

In [129]:
geoloc_mo

,name,longt,latt,coord,geodistancia
0,Arganzuela,-3.68972,40.39238,"{'type': 'Point', 'coordinates': [40.39238, -3...",0.000000
1,Arganzuela,-3.68972,40.39238,"{'type': 'Point', 'coordinates': [40.39238, -3...",0.000000
2,Arganzuela,-3.68972,40.39238,"{'type': 'Point', 'coordinates': [40.39238, -3...",0.000000
3,Arganzuela,-3.68972,40.39238,"{'type': 'Point', 'coordinates': [40.39238, -3...",0.000000
4,Arganzuela,-3.68972,40.39238,"{'type': 'Point', 'coordinates': [40.39238, -3...",0.000000
...,...,...,...,...,...
121,Barajas,-3.59395,40.46079,"{'type': 'Point', 'coordinates': [40.46079, -3...",11.116644
122,Barajas,-3.59395,40.46079,"{'type': 'Point', 'coordinates': [40.46079, -3...",11.116644
123,Barajas,-3.59395,40.46079,"{'type': 'Point', 'coordinates': [40.46079, -3...",11.116644
124,Barajas,-3.59395,40.46079,"{'type': 'Point', 'coordinates': [40.46079, -3...",11.116644
